## <div style="text-align: right">   第5章 </div>

##  <div style="text-align: right"> 进程皆有文件描述符 </div>

___

如同pid代表运行的进程，文件描述符代表打开的文件。

### 5.1 万物皆为文件

Unix哲学指出，在Unix世界中，万物皆为文件。这意味着可以将设置为视为文件，
将套接字视为文件，将文件也视为文件。

因为所有一切都被视为文件，所以当 **讨论一般意义上的文件时（包括设备、管道、套接字）时，我将使用“资源”这个词，而在表示传统定义（文件系统中的文件）时，将使用“文件”一词**。

### 5.2 描述符代表资源

无论何时在进程中打开一个资源，你都会得到一个文件描述符编号（file
description number）。文件描述符并不会在无法进程之间共享，它只存在与
其所属的进程之中。当进程结束之后，会和其他由进程打开的资源一同被关闭。
当衍生出一个进程时，文件描述符共享会有一些特殊的含义，更多内容随后详述。

在python中，进程的open_files()方法返回所有在该进程下打开的常规文件对象。返回的文件对象包括如下属性：
+ path：文件的绝对路径
+ fd：文件描述符; 在windows系统上，该值总是-1。

对于Linux系统，还支持如下属性：
+ position(Linux): 文件的位置
+ mode（Linux）:文件打开模型，与python open函数mode参数相似。
+ flags（Linux)

> 由于jupyter-notebook进程在运行时会默认打开一系列文件，不利用后续
内容讲解，因此本节中的程序均在pyhton命令行中执行

![open file](figure/open_file0.png)

上述命令显示开启一个新的python进程时，当前进程下没有任何所属的已经打开的文件。当打开passwd文件后，当前进程下添加了一个所属进程资源/etc/passwd，其文件描述符为3。 

进程打开的所有资源都会获得一个用于标识的唯一数字。这便是内核跟踪进程所用
资源的方法。

当打开多个资源的时候会怎么样呢？

![open file1](figure/open_file1.png)

这个例子有两个关键之处。
1. 所分配的文件描述符编号是尚未使用的最小的数值。我们打开的第一个文件passwd，获得的文件描述符是3,接下来打开的文件获得的文件描述符是4,这是因为3号描述符已经被使用了。

2. 资源一旦被关闭，对应的文件描述符就会被释放，随后打开的文件便可使用该描述符，当打开一个新test.txt文件时，其文件描述符为刚被释放的3。

需要注意，文件描述符只是用来跟踪打开的资源，已经关闭的资源是没有文件描述符的。

从内核的角度来看，此举意义重大。一旦资源被关闭，它就不再需要同硬件层打交道了，
因此内核也就无需再对其进行跟踪。

鉴于以上的讨论，文件描述符有时候也被称作“打开文件描述符”。这样的叫法有点用词不当，因为并没有所谓“关闭文件描述符”。

你可能已经注意到了，当我们打开一个文件，然后查询它的文件描述符时，得到的最小值是3,那0、1、2都到哪去了呢？

### 5.3 标准流

每个unix进程都有三个打开的资源，它们是标准输入（STDIN)、标准输出（STDOUT)、和
标准错误（STDERR)。它们的文件描述符分别为0、1、2。

这些标准资源是因为一个很重要的原因而存在，而如今我们已经把这个原因视为理所当然。STDIN提供了一种从键盘或管道中读取输入的通用方法，STDOUT和STDERR提供了一种向显示器、文件、打印机等输出写入内容的通用方法。这是Unix的一个创新。

在STDIN出现之前，为了能够支持键盘，你得在程序中列入一个键盘驱动程序！而且如果想在屏幕上显示一些信息，你还得知道如何控制所需的屏幕像素。所以我们要感谢标准流。

### 5.4 实践领域

文件描述符是使用套接字、管道等惊醒网络编程的核心所在，也是文件操作系统的核心。

因此，每个运行中的进程都在使用文件描述符，它们也是你通过计算机来实
现大多数趣事之关键所在。在之后的Spyglass项目中，你会看到更多例子揭示如何
使用文件描述符。

### 5.5 系统调用

在linux系统中，处理IO任务的系统调用包括 _open(2)_、_close(2)_、_read(2)_、 _write(2)_、 _pipe(2)_、 _fsync(2)_、 _stat(2)_。